In [2]:
import pandas as pd
import requests
from tqdm import tqdm
import csv

In [13]:
def download_issues(repo_owner, repo_name, label, token=None):
    issues = []
    page = 1
    headers = {}
    if token:
        headers = {
            "Authorization": f"Token {token}"
        }
    with tqdm(total=None, desc="Fetching Issues") as pbar:
        while True:
            url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/issues?state=closed&labels={label}&exclude=pulls&page={page}&per_page=100"
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                data = response.json()
                if not data:
                    break
                #IssueID: id,  IssueTitle: title, IssueBody: body, CreationTimestamp: created_at, Comment Count: comments
                issues.extend([{"IssueNumber": issue["number"], "IssueTitle": issue["title"], "IssueBody": issue["body"], "CreationTimestamp": issue["created_at"] , "CommentCount": issue["comments"]} for issue in data if "pull_request" not in issue])
                pbar.update(len(data))
                page += 1
            else:
                break
        return issues

In [14]:
repo_owner = "ansible"
repo_name = "ansible"
token = 'add_token_here'
issues = download_issues(repo_owner, repo_name,  label='bug', token=token)
print("Number of issues:", len(issues))

Fetching Issues: 39186it [19:25, 33.62it/s]

Number of issues: 18846


In [ ]:
#IssueID: id,  IssueTitle: title, IssueBody: body, CreationTimestamp: created_at, Comment Count:comments

In [ ]:
df = pd.DataFrame(issues)
df.to_csv('../data/github/github_issues.csv', encoding="utf-8")

In [16]:
df = pd.DataFrame(issues)
df.to_feather('../data/github/github_issues.feather')

In [3]:
ndf = pd.read_feather('../data/github/github_issues.feather')

In [6]:
ndf.loc[37, 'IssueBody']

"### Summary\r\n\r\nWe used `ansible.builtin.meta: refresh_inventory` extensivly to remove hosts added by host_add.  I see in the changelog that was intentional.  What's the best way to remove hosts from within a playbook now?\r\n\r\n### Issue Type\r\n\r\nBug Report\r\n\r\n### Component Name\r\n\r\nansible.builtin.meta\r\n\r\n### Ansible Version\r\n\r\n```console\r\n$ ansible --version\r\nansible [core 2.14.2]\r\n  config file = None\r\n  configured module search path = ['/home/chylarides/.ansible/plugins/modules', '/usr/share/ansible/plugins/modules']\r\n  ansible python module location = /usr/local/lib/python3.10/dist-packages/ansible\r\n  ansible collection location = /home/chylarides/.ansible/collections:/usr/share/ansible/collections\r\n  executable location = /usr/local/bin/ansible\r\n  python version = 3.10.6 (main, Nov 14 2022, 16:10:14) [GCC 11.3.0] (/usr/bin/python3)\r\n  jinja version = 3.0.3\r\n  libyaml = True\r\n```\r\n\r\n\r\n### Configuration\r\n\r\n```console\r\n# if u

In [7]:
ndf.loc[48, 'IssueTitle'][:50]

'Ansible suddenly changes SSH user in the middle of'

In [8]:
ndf['IssueBody'] = ndf['IssueBody'].str.slice(stop=30000)

In [9]:
ndf['IssueTitle'] = ndf['IssueTitle'].apply(lambda x: "`" + x if x is not None else '')
ndf['IssueBody'] = ndf['IssueBody'].apply(lambda x: "`" + x if x is not None else '')

In [169]:
ndf['IssueTitle'] = ndf['IssueTitle'].apply(lambda x: "'" + x + "'" if x.startswith('-') else x)

In [10]:
ndf.to_csv('../data/github/github_issues.csv')